In [6]:
### Name : Hemant Gangwar
### Course : B.Tech
### Intern at Linux World Informatics

In [1]:
from keras.applications import vgg16

Using TensorFlow backend.


In [2]:
img_rows = 224
img_cols = 224 
model = vgg16.VGG16(weights="imagenet", include_top = False, input_shape = (img_rows, img_cols, 3))

In [3]:
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [4]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

# Re-loads the VGG16 model without the top or FC layers
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)
    

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [5]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [6]:
model.input

<tf.Tensor 'input_2:0' shape=(None, 224, 224, 3) dtype=float32>

In [7]:
model.layers

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 2

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs=model.input, outputs=FC_Head)

print(modelnew.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
# import glob,os

In [15]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/home/spidohemant/Pictures/Images/MyPhotos/train/'
validation_data_dir = '/home/spidohemant/Pictures/Images/MyPhotos/test/'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 135 images belonging to 2 classes.
Found 38 images belonging to 2 classes.


In [16]:
r=validation_generator.class_indices
s=train_generator.class_indices
print(r)
print(s)

{'akshay': 0, 'hemant': 1}
{'akshay': 0, 'hemant': 1}


In [22]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("facerecog_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 135
nb_validation_samples = 38
epochs = 5
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

modelnew.save("facerecog_vgg.h5")

Epoch 1/5
8/8 [==============================] - 40s 5s/step - loss: 1.3369 - accuracy: 0.5798 - val_loss: 0.3597 - val_accuracy: 0.9000

Epoch 00001: val_loss improved from inf to 0.35967, saving model to facerecog_vgg.h5
Epoch 2/5
8/8 [==============================] - 39s 5s/step - loss: 0.7640 - accuracy: 0.6134 - val_loss: 0.3628 - val_accuracy: 0.8889

Epoch 00002: val_loss did not improve from 0.35967
Epoch 3/5
8/8 [==============================] - 39s 5s/step - loss: 0.5933 - accuracy: 0.6891 - val_loss: 0.1557 - val_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.35967 to 0.15568, saving model to facerecog_vgg.h5
Epoch 4/5
8/8 [==============================] - 40s 5s/step - loss: 0.6281 - accuracy: 0.6953 - val_loss: 1.0726 - val_accuracy: 0.3333

Epoch 00004: val_loss did not improve from 0.15568
Epoch 5/5
8/8 [==============================] - 39s 5s/step - loss: 0.4355 - accuracy: 0.7563 - val_loss: 1.0651 - val_accuracy: 0.7000

Epoch 00005: val_loss did not impr

In [2]:
from keras.models import load_model

Using TensorFlow backend.


In [3]:
classifier = load_model('facerecog_vgg.h5')

In [7]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

face_recogs_dict = {"[0]": "hemant gangwar ", 
                      "[1]": "akshay kumar "
                    }
face_recogs_dict_n = {"hemant": "hemant gangwar ", 
                      "akshay": "akshay kumar "
                      }
def draw_test(name, pred, im):
    person = face_recogs_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, person, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)
    
def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + face_recogs_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,3):    
    input_im = getRandomImage("/home/spidohemant/Pictures/Images/MyPhotos/test/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(1000)

cv2.destroyAllWindows()

Class - akshay kumar 
Class - akshay kumar 
Class - hemant gangwar 
